### data cleaning

In [2]:
import pandas as pd
import numpy as np

In [3]:
yanki_df = pd.read_csv(r'dataset\raw_data\yanki_ecommerce.csv')

In [4]:
yanki_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order_ID            1000 non-null   object 
 1   Customer_ID         1000 non-null   object 
 2   Customer_Name       1000 non-null   object 
 3   Product_ID          1020 non-null   object 
 4   Product_Name        1020 non-null   object 
 5   Brand               1020 non-null   object 
 6   Category            1020 non-null   object 
 7   Price               1020 non-null   float64
 8   Quantity            1020 non-null   int64  
 9   Total_Price         1020 non-null   float64
 10  Order_Date          1020 non-null   object 
 11  Shipping_Address    1020 non-null   object 
 12  City                1020 non-null   object 
 13  State               1019 non-null   object 
 14  Country             1020 non-null   object 
 15  Postal_Code         1020 non-null   int64  
 16  Email 

In [5]:
yanki_df.dropna(subset= ['Customer_ID', 'Order_ID', 'State'], inplace=True)

In [6]:
yanki_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 2 to 1019
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order_ID            999 non-null    object 
 1   Customer_ID         999 non-null    object 
 2   Customer_Name       999 non-null    object 
 3   Product_ID          999 non-null    object 
 4   Product_Name        999 non-null    object 
 5   Brand               999 non-null    object 
 6   Category            999 non-null    object 
 7   Price               999 non-null    float64
 8   Quantity            999 non-null    int64  
 9   Total_Price         999 non-null    float64
 10  Order_Date          999 non-null    object 
 11  Shipping_Address    999 non-null    object 
 12  City                999 non-null    object 
 13  State               999 non-null    object 
 14  Country             999 non-null    object 
 15  Postal_Code         999 non-null    int64  
 16  Email       

In [7]:
yanki_df['Order_Date'] = pd.to_datetime(yanki_df['Order_Date'], dayfirst=True)


In [8]:
yanki_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 2 to 1019
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Order_ID            999 non-null    object        
 1   Customer_ID         999 non-null    object        
 2   Customer_Name       999 non-null    object        
 3   Product_ID          999 non-null    object        
 4   Product_Name        999 non-null    object        
 5   Brand               999 non-null    object        
 6   Category            999 non-null    object        
 7   Price               999 non-null    float64       
 8   Quantity            999 non-null    int64         
 9   Total_Price         999 non-null    float64       
 10  Order_Date          999 non-null    datetime64[ns]
 11  Shipping_Address    999 non-null    object        
 12  City                999 non-null    object        
 13  State               999 non-null    object        
 14